In [1]:
import pandas as pd
import numpy as np
import os

---
---
Leemos las extracciones y las juntamos en un solo dataframe.

In [2]:
extraccion = pd.DataFrame()

for i in os.listdir('/content/drive/MyDrive/TFM/TFM/Data/'):
  ruta = '/content/drive/MyDrive/TFM/TFM/Data/' + i
  temp = pd.read_csv(ruta, sep="\t")
  extraccion = pd.concat([extraccion, temp], ignore_index=True)

<ipython-input-2-5ba7e8d7e30d>:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(ruta, sep="\t")
<ipython-input-2-5ba7e8d7e30d>:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(ruta, sep="\t")
<ipython-input-2-5ba7e8d7e30d>:5: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(ruta, sep="\t")
<ipython-input-2-5ba7e8d7e30d>:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(ruta, sep="\t")
<ipython-input-2-5ba7e8d7e30d>:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(ruta, sep="\t")


In [3]:
# Quitamos duplicados
extraccion.drop('S.No', axis=1, inplace = True)
extraccion_dedupped = extraccion.drop('Sample_ID', axis=1).drop_duplicates()

print(extraccion.shape)
print(extraccion_dedupped.shape)

(5133, 2667)
(5132, 2666)


In [4]:
extraccion_temp = extraccion.drop('Sample_ID', axis=1)
duplicadas_sin_sample_id = extraccion_temp.duplicated()
extraccion = extraccion[~duplicadas_sin_sample_id]

In [5]:
print(extraccion.dtypes.head(11))

Sample_ID                     object
Collection                    object
Project_ID                    object
Tissue_or_organ_of_origin     object
Tissue_subregion              object
Cell_Line                    float64
Disease                       object
Organism                      object
Gender                        object
Health_state                  object
Tissue_definition             object
dtype: object


In [6]:
# Porcentajes de N/A
cols = ['Sample_ID', 'Collection', 'Project_ID', 'Tissue_or_organ_of_origin',
       'Tissue_subregion', 'Cell_Line', 'Disease', 'Organism', 'Gender',
      'Health_state', 'Tissue_definition']
for col in cols:
    pct_missing = np.mean(extraccion[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100,2)))

Sample_ID - 0.0%
Collection - 0.0%
Project_ID - 0.0%
Tissue_or_organ_of_origin - 0.0%
Tissue_subregion - 95.46%
Cell_Line - 100.0%
Disease - 2.92%
Organism - 0.0%
Gender - 4.35%
Health_state - 2.96%
Tissue_definition - 9.04%


In [7]:
# Vemos que todas las muestras son de personas.
extraccion['Organism'].value_counts(dropna=False)

Organism
Homo sapiens    5132
Name: count, dtype: int64

In [8]:
# Observamos los distintos tipos de enfermedades
print(extraccion['Disease'].value_counts(dropna=False))

Disease
Adenocarcinoma                   1401
Squamous cell carcinoma           888
Papillary adenocarcinoma          445
Acute myeloid leukemia            188
Transitional cell carcinoma       168
                                 ... 
Astrocytoma                         1
Oligodendroglioma, anaplastic       1
Myxoid leiomyosarcoma               1
Spindle cell melanoma, type b       1
Carcinoma, undifferentiated         1
Name: count, Length: 103, dtype: int64


In [9]:
# Eliminamos columnas que no queremos
extraccion.drop(['Collection','Project_ID','Tissue_or_organ_of_origin','Tissue_subregion','Cell_Line','Organism','Gender','Tissue_definition'], axis=1, inplace=True)

In [10]:
print(extraccion.shape)
print(extraccion['Health_state'].value_counts(dropna=False))

(5132, 2659)
Health_state
disease    4594
healthy     386
NaN         152
Name: count, dtype: int64


In [11]:
extraccion = extraccion.dropna(subset=['Disease', 'Health_state']).reset_index(drop=True)

In [12]:
# Creamos dataset adenocarcinoma y guardamos los dos datasets.
extraccion = extraccion.reset_index(drop=True)
adenocarcinoma = []
for i in range(len(extraccion)):
  if 'adenocarcinoma' in extraccion['Disease'][i].lower() and extraccion['Health_state'][i] == 'disease':
    adenocarcinoma.append(True)
  else:
    adenocarcinoma.append(False)

extraccionadenocarcinoma = extraccion.copy()
extraccionadenocarcinoma['Adenocarcinoma'] = adenocarcinoma
extraccionadenocarcinoma.drop(['Disease','Health_state'], axis=1, inplace=True)

In [13]:
ruta = '/content/drive/MyDrive/TFM/TFM/Processed_Data/'
extraccionadenocarcinoma.to_csv(ruta + 'DataSetAdenocarcinoma.csv', sep='\t', index=False)

In [15]:
extraccion = pd.get_dummies(extraccion, columns = ['Health_state'])
extraccion.drop(['Disease','Health_state_healthy'], axis=1, inplace=True)

In [16]:
ruta = '/content/drive/MyDrive/TFM/TFM/Processed_Data/'
extraccion.to_csv(ruta + 'DataSet.csv', sep='\t', index=False)